<img width="8%" alt="GitHub.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/GitHub.png" style="border-radius: 15%">

# GitHub - Get comments and reactions from issue
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=GitHub+-+Get+comments+and+reactions+from+issue:+Error+short+description">Bug report</a>

**Tags:** #github #reaction #issue #comment #automation

**Author:** [Varsha Kumar](https://www.linkedin.com/in/varsha-kumar-590466305/)

**Last update:** 2024-06-05 (Created: 2024-06-04)

**Description:** This notebook allows users to retrieve the comments and reactions from an issue.

## Input

### Import libraries

In [ ]:
import requests
import re
import pandas as pd
import naas_python

### Setup variables
- `github_token`: personal token creates
- `issue_url`: link to the chosen issue

In [ ]:
github_token = naas_python.secret.get("GITHUB_TOKEN").value
issue_url = "https://github.com/pola-rs/polars/issues/16661"

## Model
### Get comments and reactions from issue

In [ ]:
def extract_repo_details(issue_url):
    # Extract owner, repo name, and issue number from the issue URL
    match = re.match(r'https://github\.com/(?P<owner>[^/]+)/(?P<repo>[^/]+)/issues/(?P<number>\d+)', issue_url)
    if not match:
        raise ValueError("Invalid issue URL")
    return match.group('owner'), match.group('repo'), match.group('number')

def get_issue_comments(repo_owner, repo_name, issue_number):
    # GitHub API URL to get issue comments
    api_url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/issues/{issue_number}/comments"
    headers = {
        'Authorization': f'token {github_token}',
        'Accept': 'application/vnd.github.v3+json'
    }

    response = requests.get(api_url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch comments: {response.status_code}")
        return []

def get_comment_reactions(repo_owner, repo_name, comment_id):
    api_url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/issues/comments/{comment_id}/reactions"
    headers = {
        'Authorization': f'token {github_token}',
        'Accept': 'application/vnd.github.squirrel-girl-preview+json'  # Required for reactions API
    }

    response = requests.get(api_url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch reactions for comment {comment_id}: {response.status_code}")
        return []

## Output
### Display result

In [ ]:
data = []

try:
    repo_owner, repo_name, issue_number = extract_repo_details(issue_url)
    comments = get_issue_comments(repo_owner, repo_name, issue_number)

    for comment in comments:
        object_url = comment['issue_url']
        comment_id = comment['id']
        comment_body = comment['body']
        comment_created_at = comment['created_at']
        comment_user_login = comment['user']['login']
        comment_user_id = comment['user']['id']
        
        data = [{
            "OBJECT_URL": object_url,
            "ID": comment_id,
            "TYPE": "COMMENT",
            "CONTENT": comment_body,
            "DATE_TIME": comment_created_at,
            "USER_LOGIN": comment_user_login,
            "USER_ID": comment_user_id,
        }]
        
        reactions = get_comment_reactions(repo_owner, repo_name, comment_id)
        for reaction in reactions:
            reaction_id = reaction['id']
            reaction_body = reaction['content']
            reaction_created_at = reaction['created_at']
            reaction_user_login = reaction['user']['login']
            reaction_user_id = reaction['user']['id']
            
            data.append({
                "OBJECT_URL": object_url,
                "ID": comment_id,
                "TYPE": "REACTION",
                "CONTENT": reaction_body,
                "DATE_TIME": reaction_created_at,
                "USER_LOGIN": reaction_user_login,
                "USER_ID": reaction_user_id,
            })
        
        

except ValueError as e:
    print(e)
    
df = pd.DataFrame(data)
df